# Model Exploration Dash Prototype 

In [15]:
## Imports
#

import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import callback_context
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import os, ntpath
import numpy as np
from functools import reduce

In [16]:
## Defaults
#

default_num_groups = 5
default_trajectory_selector = 'slider' # either 'dropdown' or 'slider'

In [17]:
## Helpers
#

def options_factory(iterable, label_accessor=lambda x: x, value_accessor=lambda x: x):
    """
    Factory function for creating a dash dropdown menu's 'options' attribute.

    Positional arguments:
    iterable -- any iterable data structure.

    Keyword arguments (optional):
    label_accessor -- function to retrieve the label from a single data point in iterable.
    value_accessor -- function to retrieve the value from a single data point in iterable.

    Returns a list of dictionaries with keys 'label' and 'value'.
    """
    return [
        { 'label': label_accessor(d), 'value': value_accessor(d) }
        for d in iterable
    ]


def read_ensemble_data_from_results_folder(folder):
    """
    Read GillesPy2 ensemble simulation results into a list of pandas data frames.
    
    Positional arguments:
    folder -- folder on disk containing csv files with one trajectory result per file.
    
    Returns:
    List of pandas DataFrame objects where the index represents the trajectory number and 
    values are 2-dimensional arrays where columns are species and rows are 
    result values for each species at some time step.
    """
    files = sorted([ os.path.join(results_dir, f) for f in os.listdir(results_dir) ])
    dfs = [ pd.read_csv(f_path) for f_path in files ]
    return dfs


def get_zeroed_df(df):
    zero_df = df - df
    zero_df['time'] = df['time'].copy()
    return zero_df


def compute_dfs_mean(dataframes):
    """
    Compute the mean dataframe of a list of pandas dataframes.
    """
    if len(dataframes):
        # Using overloaded pandas DataFrame operators
        sum = reduce(lambda df1, df2: df1 + df2, dataframes)
        mean = sum / len(dataframes)
        return mean.to_json(orient='split')
    else:
        # Empty Group
        return get_zeroed_df(dfs[0]).to_json(orient='split')


def get_group_trajectories(group, group_assigns):
    """
    Given a group index and list of group assignments,
    return a list of trajectory indices belonging to that group.
    
    group -- group number (0-indexed)
    group_assigns -- a list where an index is a trajectory id and a value is
                     the group the trajectory is assigned to.
    """
    return [ i for i in range(len(group_assigns)) if group_assigns[i] == group ]


def compute_group_data(group_data, group_assigns, dfs):
    """
    Compute all group metrics based on current group assignments
    """
    # if group has no data and now has data, we'll need to make a new data frame
    dfs = [ pd.read_json(df, orient='split') for df in dfs ]
    for group in range(len(group_data)):
        group_trjs = get_group_trajectories(group, group_assigns)
        group_dfs = [ dfs[i] for i in range(len(dfs)) if i in group_trjs ]
        group_data[group] = compute_dfs_mean(group_dfs)
    return group_data


def default_local_store(dfs, num_groups):
    """
    Initial client-side JSON storage for callbacks to refernece.
    """
    return {
        'dfs': [ df.to_json(orient='split') for df in dfs ],
        'group_assigns' : [None] * len(dfs),
        'group_data' : [get_zeroed_df(dfs[0]).to_json(orient='split')] * default_num_groups
    }

In [18]:
## Read the data
#

results_dir = './results_csv_06042020_150109/'
dfs = read_ensemble_data_from_results_folder(results_dir)

In [19]:
## Setup menus and styling for dropdowns
#

# Extra styling for dropdowns
dropdown_styles = {
}

# Setup species dropdown options
all_species = [ c for c in dfs[0].columns if c != 'time' ]
species_options = options_factory(all_species)
species_options.insert(0, options_factory(['All'])[0])

# Group dropdown options
group_options = options_factory(
    # 0-indexed internally, labeled as 1-indexed
    range(default_num_groups), label_accessor=lambda x: str(x+1)
)

# Trajectory dropdown options
trajectory_options = options_factory(
    # 0-indexed internally, labeled as 1-indexed
    range(len(dfs)), label_accessor=lambda x: str(x+1)
)

In [20]:
## App
#

external_stylesheets = [
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [21]:
## Fragments
#

# Initial client JSON store
store = default_local_store(dfs, default_num_groups)

# Trajectory selector (dropdown)
trajectory_selector_dropdown = html.Label([
    dcc.Dropdown(style=dropdown_styles,
        id='trajectory-selector', clearable=False,
        options=trajectory_options, value=0)
])

# Trajectory selector (slider)
trajectory_selector_slider = html.Div([
    dcc.Slider(
        id='trajectory-selector',
        min=0,
        max=len(trajectory_options)-1,
        step=1,
        value=0,
        marks = { d['value']:d['label'] for d in trajectory_options }
    ),
    html.Div(id='slider-output-container')
])

# Get default trajectory selector (slider or dropdown, determined from config above)
default_trajectory_selector = trajectory_selector_dropdown if default_trajectory_selector == 'dropdown' else trajectory_selector_slider

In [22]:
## Components
#

In [23]:
## Group Assigner layout
group_assigner_layout = html.Div([
    html.Div([
        html.H4(children="Assign Ensemble Trajectories to Groups")
    ], className='col-md-12'),
    html.Div([
        html.Label("Trajectory"),
        html.Div(children=default_trajectory_selector, id='trajectory-selector-container'),
        html.Button('Dropdown', id='select-trajectory-dropdown-btn', n_clicks=0),
        html.Button('Slider', id='select-trajectory-slider-btn', n_clicks=0)
        ],
        id='trajectory-selector-wrapper',
        className='col-md-12'
    ),
    html.Div([
        html.Label("Show Species"),
        dcc.Dropdown(
                id='species-dropdown-trajectory',
                style=dropdown_styles,
                clearable=False,
                value='All',
                options=species_options
        )
        ],
        id='group-assigner-select-species-wrapper',
        className='col-md-6'
    ),
    html.Div([
        html.Label("Assign to Group"),
        dcc.Dropdown(id='group-assigner-group-select-dropdown', clearable=True, style=dropdown_styles, options=group_options, value=0)
        ],
        id='group-assigner-dropdown-wrapper',
        className='col-md-6'
    ),
    html.Div([
        html.Div([
            html.Button('Toggle Graph Display', id='toggle-trajectory-graph-btn', n_clicks=0,
            style={ 'padding': '5px', 'margin-top': '10px' }),
        ]
        ),
        html.Div([ dcc.Graph(id='group-assigner-graph') ],
                 id='group-assigner-graph-wrapper',
                 className='col-md-6',
        ),
        ],
        id='group-assigner-graph-and-tgl-btn-wrapper',
        className='col-md-12'
    )
], className='card card-body')


In [24]:
## Group assigner callbacks

# Update the trajectory graph
@app.callback(
    Output('group-assigner-graph', 'figure'),
    [Input('species-dropdown-trajectory', 'value'),
     Input('trajectory-selector', 'value')]
)
def update_group_assignment_graph(species, trajectory):
    if species == 'All':
        species = all_species
    return px.line(
        dfs[trajectory], x="time", y=species,
        render_mode="webgl", title="Trajectory {}".format(trajectory+1)
    )


# Toggle the trajectory graph display
@app.callback(
    Output('group-assigner-graph', 'style'),
    [Input('toggle-trajectory-graph-btn', 'n_clicks')],
)
def toggle_group_assignment_graph(n_clicks):
    changed_id = [p['prop_id'] for p in callback_context.triggered][0]
    if 'toggle-trajectory-graph-btn' in changed_id and n_clicks:
        return { 'display' : 'none' } if n_clicks % 2 else {}
    return {}


# Switch trajectory selector interface (slider or dropdown)
@app.callback(
    Output('trajectory-selector-container', 'children'),
    [Input('select-trajectory-dropdown-btn', 'n_clicks'),
     Input('select-trajectory-slider-btn', 'n_clicks')],
    [State('trajectory-selector-container', 'children')]
)
def update_trajectory_selector_dropdown(dropdown_n_clicks, slider_n_clicks, current):
    changed_id = [p['prop_id'] for p in callback_context.triggered][0]
    if 'select-trajectory-dropdown-btn' in changed_id and dropdown_n_clicks:
        return trajectory_selector_dropdown
    if 'select-trajectory-slider-btn' in changed_id and slider_n_clicks:
        return trajectory_selector_slider
    return current


# Update the 'set group' dropdown when selected trajectory changes
@app.callback(
    Output('group-assigner-group-select-dropdown', 'value'),
    [Input('memory', 'modified_timestamp'),
     Input('trajectory-selector', 'value')],
    [State('memory', 'data')]
)
def update_group_dropdown(timestamp, trajectory, memory):
    return memory['group_assigns'][trajectory]


# Store the trajectory's group when the user selects from the group dropdown
# Recompute group metrics and update group data
# TODO the group dropdown changes when the trajectory is changed too
#      is this a memory input?
@app.callback(
    Output('memory', 'data'),
    [Input('group-assigner-group-select-dropdown', 'value')],
    [State('trajectory-selector', 'value'),
     State('memory', 'data')]
)
def update_trajectory_group(selected_group, trajectory, memory):
    from_group = memory['group_assigns'][trajectory]
    memory['group_assigns'][trajectory] = selected_group
    if from_group != selected_group:
        memory['group_data'] = compute_group_data(memory['group_data'], memory['group_assigns'], memory['dfs'])
    return memory


In [25]:
## Group inspector layout
group_inspector_layout = html.Div([
    html.H4(children="Group Inspector"),
    html.Label([
    "Select Group",
    dcc.Dropdown(id='group-inspector-select-group-dropdown', clearable=True, style=dropdown_styles, options=group_options, value=0)
    ]),
    html.Label([
    "Show Species",
    dcc.Dropdown(style=dropdown_styles,
        id='group-inspector-species-dropdown', clearable=False,
        value='All', options=species_options)
    ]),
    dcc.Graph(id='group-inspect-graph')
], className='card card-body')

In [26]:
## Group inspector callbacks

# Update the group inspector graph
@app.callback(
    Output('group-inspect-graph', 'figure'),
    [Input('group-inspector-select-group-dropdown', 'value'),
     Input('group-inspector-species-dropdown', 'value'),
     Input('memory', 'data')]
)
def update_group_inspector_graph(group, species, memory):
    if species == 'All':
        species = all_species
    try:
        return px.line(pd.read_json(memory['group_data'][group], orient='split'),
                       x='time', y=species, render_mode='webgl',
                       title='Group {}'.format(group+1)
                      )
    except:
        return px.line()

In [27]:
## App Layout
#

app.layout = html.Div([
    dcc.Store(id='memory', data=store),
    group_assigner_layout,
    group_inspector_layout
],
style={
    'font-family': 'Arial, Helvetica, sans-serif'
})

In [29]:
## Run
#

app.run_server(mode='inline', debug=True)